In [1]:
import torch
from utils.dataset import get_data_loaders
import hydra
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose
import torch
import os
from utils.dataset import get_data_loaders
from models.CNN import CNN
from tqdm import tqdm
from utils.dataset import get_data_loaders
from models.CNN import CNN
import pickle

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is NOT available. Using CPU.")

hydra.core.global_hydra.GlobalHydra.instance().clear()
with initialize(version_base=None, config_path="../src/configs"):
    cfg = compose(config_name="base", overrides=["project_root=/home/paperspace/DeepEmotion"])
print(cfg)

CUDA is available. Using GPU: Quadro M4000
{'project_root': '/home/paperspace/DeepEmotion', 'verbose': True, 'wandb': True, 'sys_log': True, 'model': 'CNN', 'CNN': {'c1': 16, 'c2': 32, 'c3': 64, 'k1': 3, 'k2': 3, 'k3': 3, 'pk': 2, 'ps': 2, 'kernel_size': 3, 'stride': 1, 'padding': 1}, 'train': {'epochs': 50, 'batch_size': 2, 'shuffle': True, 'train_ratio': 0.8, 'print_label_frequencies': True}, 'data': {'data_path': '${project_root}/data/raw/derivatives/non-linear_anatomical_alignment', 'zarr_dir_path': '${project_root}/zarr_datasets', 'zarr_path': '${project_root}/zarr_datasets/pool_emotions', 'label_path': '${project_root}/data/updated_annotations/pooled_annotations_structured.tsv', 'sessions': ['02'], 'file_pattern_template': '*_ses-forrestgump_task-forrestgump_rec-dico7Tad2grpbold7TadNL_run-{}_bold.nii.gz', 'subjects': ['sub-01'], 'session_offsets': [0, 902, 1784, 2660, 3636, 4560, 5438, 6522], 'emotion_idx': {'NONE': 0, 'HAPPINESS': 1, 'FEAR': 2, 'SADNESS': 3, 'LOVE': 4, 'ANGER': 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Suppose cfg.data.emotion_idx maps something like { "happy": 0, "sad": 1, ... }
# We'll create a dict to hold a list of gradients for each class:
class_gradients = {
    idx: [] for emotion, idx in cfg.data.emotion_idx.items() if emotion != "NONE"
}

# Config variables
PROJECT_ROOT = os.path.abspath("../")

cfg = OmegaConf.create({
    "project_root": PROJECT_ROOT,
    "verbose": True,
    "wandb": True,
    "sys_log": True,
    "model": "CNN",
    "CNN": {
        "c1": 16, "c2": 32, "c3": 64, "k1": 3, "k2": 3, "k3": 3,
        "pk": 2, "ps": 2, "kernel_size": 3, "stride": 1, "padding": 1
    },
    "train": {
        "epochs": 50, "batch_size": 10, "shuffle": True, "train_ratio": 0.8,
        "print_label_frequencies": True
    },
    "data": {
        "data_path": f"{PROJECT_ROOT}/data/raw/derivatives/non-linear_anatomical_alignment",
        "zarr_dir_path": f"{PROJECT_ROOT}/zarr_datasets",
        "zarr_path": f"{PROJECT_ROOT}/zarr_datasets/pool_emotions",
        "label_path": f"{PROJECT_ROOT}/data/updated_annotations/pooled_annotations_structured.tsv",
        "sessions": ["01", "02", "03", "04", "05", "06", "07", "08"],
        "file_pattern_template": "*_ses-forrestgump_task-forrestgump_rec-dico7Tad2grpbold7TadNL_run-{}_bold.nii.gz",
        "subjects": ["sub-1"],
        "session_offsets": [0, 902, 1784, 2660, 3636, 4560, 5438, 6522],
        "emotion_idx": {"NONE": 0, "HAPPINESS": 1, "FEAR": 2, "SADNESS": 3, "LOVE": 4, "ANGER": 5},
        "normalization": False,
        "weight_decay": 0,
        "learning_rate": 0.0001,
        "seed": 42,
        "save_model": True,
        "load_model": False,
        "save_model_path": "output/models",
        "load_model_path": f"{PROJECT_ROOT}/src/output/models/sub_ALL.pth",
        "output_csv_path": f"{PROJECT_ROOT}/src/output/PCA/hidden/sub_ALL.csv"
    }
})